2D AND 3D VERIFICATION AND VALIDATION OF THE LATTICE BOLTZMANN METHOD

https://publications.polymtl.ca/1927/1/2015_MatteoPortinari.pdf

Lattice Boltzmann Method Simulation of 3-D Natural Convection with Double MRT Model

https://arxiv.org/pdf/1511.04633.pdf

Multiple-relaxation-time Lattice Boltzmann Models in 3D

https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/20020075050.pdf

Multiple-relaxation-time lattice Boltzmann model for the convection and anisotropic diffusion equation

https://www.researchgate.net/publication/222659771_Multiple-relaxation-time_lattice_Boltzmann_model_for_the_convection_and_anisotropic_diffusion_equation

In [1]:
import math
import sympy
from sympy import Matrix, diag, symbols, pprint
from sympy.codegen.ast import Assignment
from sympy.printing.ccode import C99CodePrinter

sympy.init_printing(use_unicode=True, num_columns=200, wrap_line=False)

In [2]:
class MyCodePrinter(C99CodePrinter):
    def __init__(self):
        super().__init__()
        self.lines = []

    def define(self, *var, type='real'):
        for v in var:
            if isinstance(v, Matrix):
                self.lines += [
                    f'{type} {", ".join([str(v.row(i)[0]) for i in range(0, v.shape[0])])};']
            else:
                self.lines += [f'{type} {v};']

    def append(self, expr):
        self.lines += [expr]

    def let(self, var, expr):
        if isinstance(var, Matrix):
            for i in range(0, var.shape[0]):
                self.lines += [self.doprint(Assignment(var.row(i)[0], expr.row(i)[0]))]
        else:
            self.lines += [self.doprint(Assignment(var, expr))]
    
    def __repr__(self):
        return "\n".join(self.lines)

code = MyCodePrinter()

In [3]:
# Kinematic viscosity
nu = symbols('nu')
# Thermal diffusivity
nuT = symbols('nuT')
# Smagorinsky constant
C = symbols('C')
# Turbulent Prandtl number
Pr_t = symbols('Pr_t')
# Gravity times thermal expansion
gBetta = symbols('gBetta')
# Reference temperature for Boussinesq
Tref = symbols('Tref')
# Velocity PDFs
fi = Matrix([symbols(f'f{i}') for i in range(0, 19)])
# Temperature PDFs
Ti = Matrix([symbols(f'T{i}') for i in range(0, 7)])

# Mean density in system
rho_0 = 1
delta_x = 1

In [4]:
# Temporary variables
rho = symbols('rho')
T = symbols('T')
vx = symbols('vx')
vy = symbols('vy')
vz = symbols('vz')
V = Matrix([vx, vy, vz])
fi_eq = Matrix([symbols(f'f{i}eq') for i in range(0, 19)])
mi = Matrix([symbols(f'm{i}') for i in range(0, 19)])
mi_eq = Matrix([symbols(f'm{i}eq') for i in range(0, 19)])
mi_neq = Matrix([symbols(f'm{i}neq') for i in range(0, 19)])
omega = Matrix([symbols(f'omega{i}') for i in range(0, 19)])
Sxx = symbols('Sxx')
Syy = symbols('Syy')
Szz = symbols('Szz')
Sxy = symbols('Sxy')
Syz = symbols('Syz')
Sxz = symbols('Sxz')
jx = symbols('jx')
jy = symbols('jy')
jz = symbols('jz')
m1_1 = symbols('m1_1')
m1_9 = symbols('m1_9')
m1_11 = symbols('m1_11')
m1_13 = symbols('m1_13')
m1_14 = symbols('m1_14')
m1_15 = symbols('m1_15')
S_bar = symbols('S_bar')
nu_t = symbols('nu_t')

code.define(rho, T, V, fi_eq, mi, mi_eq, mi_neq, omega, Sxx, Syy, Szz, Sxy, Syz, Sxz, jx, jy, jz, m1_1, m1_9, m1_11, m1_13, m1_14, m1_15, S_bar, nu_t)

In [5]:
# LBM velocity vectors for D3Q19 (and D3Q7)
ei = Matrix([
    [0, 0, 0],
    [1, 0, 0],
    [-1, 0, 0],
    [0, 1, 0],
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, -1],
    [1, 1, 0],
    [-1, -1, 0],
    [1, -1, 0],
    [-1, 1, 0],
    [1, 0, 1],
    [-1, 0, -1],
    [1, 0, -1],
    [-1, 0, 1],
    [0, 1, 1],
    [0, -1, -1],
    [0, 1, -1],
    [0, -1, 1]
])

print(ei.shape)
pprint(ei)

(19, 3)
⎡0   0   0 ⎤
⎢          ⎥
⎢1   0   0 ⎥
⎢          ⎥
⎢-1  0   0 ⎥
⎢          ⎥
⎢0   1   0 ⎥
⎢          ⎥
⎢0   -1  0 ⎥
⎢          ⎥
⎢0   0   1 ⎥
⎢          ⎥
⎢0   0   -1⎥
⎢          ⎥
⎢1   1   0 ⎥
⎢          ⎥
⎢-1  -1  0 ⎥
⎢          ⎥
⎢1   -1  0 ⎥
⎢          ⎥
⎢-1  1   0 ⎥
⎢          ⎥
⎢1   0   1 ⎥
⎢          ⎥
⎢-1  0   -1⎥
⎢          ⎥
⎢1   0   -1⎥
⎢          ⎥
⎢-1  0   1 ⎥
⎢          ⎥
⎢0   1   1 ⎥
⎢          ⎥
⎢0   -1  -1⎥
⎢          ⎥
⎢0   1   -1⎥
⎢          ⎥
⎣0   -1  1 ⎦


In [6]:
# Density weighting factors for D3Q19 velocity PDFs
e_omega = Matrix([
    1/3,
    1/18,
    1/18,
    1/18,
    1/18,
    1/18,
    1/18,
    1/36,
    1/36,
    1/36,
    1/36,
    1/36,
    1/36,
    1/36,
    1/36,
    1/36,
    1/36,
    1/36,
    1/36
])

# Density weighting factors for D3Q7 energy PDFs
e_omegaT = Matrix([
    1/4,
    1/8,
    1/8,
    1/8,
    1/8,
    1/8,
    1/8
])

print(e_omega.shape)
pprint(e_omega)
print(e_omegaT.shape)
pprint(e_omega)

(19, 1)
⎡0.333333333333333 ⎤
⎢                  ⎥
⎢0.0555555555555556⎥
⎢                  ⎥
⎢0.0555555555555556⎥
⎢                  ⎥
⎢0.0555555555555556⎥
⎢                  ⎥
⎢0.0555555555555556⎥
⎢                  ⎥
⎢0.0555555555555556⎥
⎢                  ⎥
⎢0.0555555555555556⎥
⎢                  ⎥
⎢0.0277777777777778⎥
⎢                  ⎥
⎢0.0277777777777778⎥
⎢                  ⎥
⎢0.0277777777777778⎥
⎢                  ⎥
⎢0.0277777777777778⎥
⎢                  ⎥
⎢0.0277777777777778⎥
⎢                  ⎥
⎢0.0277777777777778⎥
⎢                  ⎥
⎢0.0277777777777778⎥
⎢                  ⎥
⎢0.0277777777777778⎥
⎢                  ⎥
⎢0.0277777777777778⎥
⎢                  ⎥
⎢0.0277777777777778⎥
⎢                  ⎥
⎢0.0277777777777778⎥
⎢                  ⎥
⎣0.0277777777777778⎦
(7, 1)
⎡0.333333333333333 ⎤
⎢                  ⎥
⎢0.0555555555555556⎥
⎢                  ⎥
⎢0.0555555555555556⎥
⎢                  ⎥
⎢0.0555555555555556⎥
⎢                  ⎥
⎢0.0555555555555556⎥
⎢                  

In [7]:
# Compute physical quantities
code.let(rho, (sympy.ones(1, 19)*fi)[0])
code.let(T, (sympy.ones(1, 7)*Ti)[0])
code.let(vx, (Matrix([ei.row(i)[0] for i in range(0, 19)]).transpose()*fi)[0])
code.let(vy, (Matrix([ei.row(i)[1] for i in range(0, 19)]).transpose()*fi)[0])
code.let(vz, (Matrix([ei.row(i)[2] for i in range(0, 19)]).transpose()*fi)[0])


In [8]:
def feq(ei):
    cs2 = 1/3  # Speed of sound squared
    return rho*e_omega*(1 + ei.dot(V)/cs2 + (ei.dot(V))**2/(2*cs2**2) - V.dot(V)/(2*cs2))


# Equilibrium PDFs in velocity space
code.let(fi_eq, Matrix([feq(ei.row(i)) for i in range(0, 19)]))

In [9]:
def phi(ei):
    """Transformation matrix for transition from velocity to moment space
    """
    p0 = ei.norm()**0
    p1 = 19*ei.norm()**2 - 30
    p2 = (21*ei.norm()**4 - 53*ei.norm()**2 + 24)/2
    p3 = ei[0]
    p4 = (5*ei.norm()**2 - 9)*ei[0]
    p5 = ei[1]
    p6 = (5*ei.norm()**2 - 9)*ei[1]
    p7 = ei[2]
    p8 = (5*ei.norm()**2 - 9)*ei[2]
    p9 = 3*ei[0]**2 - ei.norm()**2
    p10 = (3*ei.norm()**2 - 5)*(3*ei[0]**2 - ei.norm()**2)
    p11 = ei[1]**2 - ei[2]**2
    p12 = (3*ei.norm()**2 - 5)*(ei[1]**2 - ei[2]**2)
    p13 = ei[0]*ei[1]
    p14 = ei[1]*ei[2]
    p15 = ei[0]*ei[2]
    p16 = (ei[1]**2 - ei[2]**2)*ei[0]
    p17 = (ei[2]**2 - ei[0]**2)*ei[1]
    p18 = (ei[0]**2 - ei[1]**2)*ei[2]
    return [p0, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16, p17, p18]


M = Matrix([phi(ei.row(i)) for i in range(0, 19)]).transpose()

print(M.shape)
pprint(M)

(19, 19)
⎡ 1    1    1    1    1    1    1   1   1   1   1   1   1   1   1   1   1   1 
⎢                                                                             
⎢-30  -11  -11  -11  -11  -11  -11  8   8   8   8   8   8   8   8   8   8   8 
⎢                                                                             
⎢12   -4   -4   -4   -4   -4   -4   1   1   1   1   1   1   1   1   1   1   1 
⎢                                                                             
⎢ 0    1   -1    0    0    0    0   1   -1  1   -1  1   -1  1   -1  0   0   0 
⎢                                                                             
⎢ 0   -4    4    0    0    0    0   1   -1  1   -1  1   -1  1   -1  0   0   0 
⎢                                                                             
⎢ 0    0    0    1   -1    0    0   1   -1  -1  1   0   0   0   0   1   -1  1 
⎢                                                                             
⎢ 0    0    0   -4    4    0    0   1   -1 

In [10]:
def chi(ei):
    """Transformation matrix for transition from energy PDFs to moment space
    """
    p0 = ei.norm()**0
    p1 = ei[0]
    p2 = ei[1]
    p3 = ei[2]
    p4 = 6 - 7*ei.norm()**2
    p5 = 3*ei[0]**2 - ei.norm()**2
    p6 = ei[1]**2 - ei[2]**2
    return [p0, p1, p2, p3, p4, p5, p6]


N = Matrix([chi(ei.row(i)) for i in range(0, 7)]).transpose()

print(N.shape)
pprint(N)

(7, 7)
⎡1  1   1   1   1   1   1 ⎤
⎢                         ⎥
⎢0  1   -1  0   0   0   0 ⎥
⎢                         ⎥
⎢0  0   0   1   -1  0   0 ⎥
⎢                         ⎥
⎢0  0   0   0   0   1   -1⎥
⎢                         ⎥
⎢6  -1  -1  -1  -1  -1  -1⎥
⎢                         ⎥
⎢0  2   2   -1  -1  -1  -1⎥
⎢                         ⎥
⎣0  0   0   1   1   -1  -1⎦


In [11]:
def eq(m):
    """Calculate equlilibrium PDFs in moment space
    """
    rho = m[0]  # Density
    en = m[1]  # Energy
    epsilon = m[2]  # Energy square
    jx = m[3]  # Momentum
    qx = m[4]  # Energy flux
    jy = m[5]
    qy = m[6]
    jz = m[7]
    qz = m[8]
    # Symmetric viscous stress tensor
    pxx = m[9]/3
    pixx = m[10]/3
    pww = m[11]
    piww = m[12]
    pxy = m[13]
    pyz = m[14]
    pxz = m[15]
    # Antisymmetric third-order moment
    mx = m[16]
    my = m[17]
    mz = m[18]
    # Model stability constants
    omega_e = 0
    omega_xx = 0
    omega_ej = -475/63

    # Output
    meq0 = rho
    meq1 = -11*rho + 19/rho_0*(jx*jx + jy*jy + jz*jz)
    meq2 = omega_e*rho + omega_ej/rho_0*(jx*jx + jy*jy + jz*jz)
    meq3 = jx
    meq4 = -2/3*jx
    meq5 = jy
    meq6 = -2/3*jy
    meq7 = jz
    meq8 = -2/3*jz
    meq9 = (3*jx*jx - (jx*jx + jy*jy + jz*jz))/rho_0
    meq10 = omega_xx*meq9
    meq11 = (jy*jy - jz*jz)/rho_0
    meq12 = omega_xx*meq11
    meq13 = jx*jy/rho_0
    meq14 = jy*jz/rho_0
    meq15 = jx*jz/rho_0
    meq16 = 0
    meq17 = 0
    meq18 = 0
    return Matrix([meq0, meq1, meq2, meq3, meq4, meq5, meq6, meq7, meq8, meq9, meq10, meq11, meq12, meq13, meq14, meq15, meq16, meq17, meq18])


# Transform velocity PDFs to moment space
m = M*fi
code.let(mi, m)
# Velocity moments equilibirum PDFs
m_eq = eq(m)
code.let(mi_eq, m_eq)

# Strain rate tensor
code.let(jx, m[3])
code.let(jy, m[5])
code.let(jz, m[7])
code.let(m1_1, 38/3*(jx + jy + jz))
code.let(m1_9, -2/3*(2*jx - jy - jz))
code.let(m1_11, -2/3*(jy - jz))
code.let(m1_13, -1/3*(jx + jy))
code.let(m1_14, -1/3*(jz + jy))
code.let(m1_15, -1/3*(jx + jz))
code.let(Sxx, -m1_1/(38*rho_0) - m1_9/(2*rho_0))
code.let(Syy, -m1_1/(38*rho_0) + m1_9/(4*rho_0) - 3*m1_11/(4*rho_0))
code.let(Szz, -m1_1/(38*rho_0) + m1_9/(4*rho_0) + 3*m1_11/(4*rho_0))
code.let(Sxy, -3*m1_13/(2*rho_0))
code.let(Syz, -3*m1_14/(2*rho_0))
code.let(Sxz, -3*m1_15/(2*rho_0))
#code.let(S_bar, (2*(Sxx*Sxx + Syy*Syy + Szz*Szz + Sxy*Sxy + Syz*Syz + Sxz*Sxz))**(1/2))
code.let(S_bar, (Sxx*Sxx + Syy*Syy + Szz*Szz + 2*Sxy*Sxy + 2*Syz*Syz + 2*Sxz*Sxz)**(1/2))
# Eddy viscosity
code.let(nu_t, (C*delta_x)**2*S_bar)

In [12]:
# Collision matrix for velocities in moment space
s1 = s4 = s6 = s8 = 0
s2 = 1.19
s3 = s11 = s13 = 1.4
s5 = s7 = s9 = 1.2
s17 = s18 = s19 = 1.98
s10 = s12 = s14 = s15 = s16 = 2/(1 + 6*(nu + nu_t))
S_hat = sympy.diag(s1, s2, s3, s4, s5, s6, s7, s8, s9, s10,
                   s11, s12, s13, s14, s15, s16, s17, s18, s19)

print(S_hat.shape)
pprint(S_hat)

(19, 19)
⎡0   0     0   0   0   0   0   0   0         0          0         0          0
⎢                                                                             
⎢0  1.19   0   0   0   0   0   0   0         0          0         0          0
⎢                                                                             
⎢0   0    1.4  0   0   0   0   0   0         0          0         0          0
⎢                                                                             
⎢0   0     0   0   0   0   0   0   0         0          0         0          0
⎢                                                                             
⎢0   0     0   0  1.2  0   0   0   0         0          0         0          0
⎢                                                                             
⎢0   0     0   0   0   0   0   0   0         0          0         0          0
⎢                                                                             
⎢0   0     0   0   0   0  1.2  0   0       

In [13]:
# Nonequilibrium moments
code.let(mi_neq, m - m_eq)

#code.let(omega, -M**-1*(S_hat*mi_neq))

In [14]:
Phi = M*M.transpose()
assert(M*(M.transpose()*Phi**-1) == sympy.eye(19,19))
Lambda = Phi**-1*S_hat
print(Lambda.shape)
pprint(Lambda)
code.let(omega, -M.transpose()*(Lambda*mi_neq))

(19, 19)
⎡0           0                     0           0   0    0   0    0   0        
⎢                                                                             
⎢0  0.000497076023391813           0           0   0    0   0    0   0        
⎢                                                                             
⎢0           0            0.00555555555555555  0   0    0   0    0   0        
⎢                                                                             
⎢0           0                     0           0   0    0   0    0   0        
⎢                                                                             
⎢0           0                     0           0  0.03  0   0    0   0        
⎢                                                                             
⎢0           0                     0           0   0    0   0    0   0        
⎢                                                                             
⎢0           0                     0       

In [15]:
for i in range(0, 19):
    code.append(
        f'dftmp3D({i}, x, y, z, nx, ny, nz) = {fi.row(i)[0]} + {omega.row(i)[0]};')

for i in range(0, 7):
    code.append(
        f'Tdftmp3D({i}, x, y, z, nx, ny, nz) = {Ti.row(i)[0]};')

print(code)

real rho;
real T;
real vx, vy, vz;
real f0eq, f1eq, f2eq, f3eq, f4eq, f5eq, f6eq, f7eq, f8eq, f9eq, f10eq, f11eq, f12eq, f13eq, f14eq, f15eq, f16eq, f17eq, f18eq;
real m0, m1, m2, m3, m4, m5, m6, m7, m8, m9, m10, m11, m12, m13, m14, m15, m16, m17, m18;
real m0eq, m1eq, m2eq, m3eq, m4eq, m5eq, m6eq, m7eq, m8eq, m9eq, m10eq, m11eq, m12eq, m13eq, m14eq, m15eq, m16eq, m17eq, m18eq;
real m0neq, m1neq, m2neq, m3neq, m4neq, m5neq, m6neq, m7neq, m8neq, m9neq, m10neq, m11neq, m12neq, m13neq, m14neq, m15neq, m16neq, m17neq, m18neq;
real omega0, omega1, omega2, omega3, omega4, omega5, omega6, omega7, omega8, omega9, omega10, omega11, omega12, omega13, omega14, omega15, omega16, omega17, omega18;
real Sxx;
real Syy;
real Szz;
real Sxy;
real Syz;
real Sxz;
real jx;
real jy;
real jz;
real m1_1;
real m1_9;
real m1_11;
real m1_13;
real m1_14;
real m1_15;
real S_bar;
real nu_t;
rho = f0 + f1 + f10 + f11 + f12 + f13 + f14 + f15 + f16 + f17 + f18 + f2 + f3 + f4 + f5 + f6 + f7 + f8 + f9;
T = T0 + T1 + T2 